In [1]:
import pandas as pd
import us
import numpy as np
import geopandas as gpd

# Loading the Main Data

In [2]:

#Load Data
earthquake = pd.read_csv('data/Eartquakes-1990-2023.csv')
realtor = pd.read_csv('data/realtor-data.csv')

#Clean up empty spaces
earthquake['state'] = earthquake['state'].str.strip()

#Change USA to the correct state
earthquake['state'] = earthquake['state'].replace('USA', 'Georgia')

# US states 
states = us.states.STATES
List = []
for state in states:
    List.append(state.name)
    List.append(state.abbr)

#Filter for US states 
filtered_earthquake = earthquake[earthquake['state'].isin(List)]
filtered_realtor = realtor[realtor['state'].isin(List)]

#Change all abbreviation to full name (only for earthquake)
    # Dictionary mapping abbreviations to full names
us_states = {state.abbr: state.name for state in states}
filtered_earthquake.loc[:, 'state'] = filtered_earthquake['state'].apply(lambda x: us_states.get(x, x))

#Drop NaN from both dataframe
filtered_earthquake = filtered_earthquake.dropna()
filtered_realtor = filtered_realtor.dropna()

# filtered_earthquake
filtered_realtor

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
3409,21163.0,for_sale,525000.0,3.0,3.0,0.45,1813270.0,Agawam,Massachusetts,1001.0,2314.0,2014-06-25
3410,67455.0,for_sale,289900.0,3.0,2.0,0.36,1698080.0,Agawam,Massachusetts,1001.0,1276.0,2012-10-12
3416,97400.0,for_sale,384900.0,3.0,2.0,0.46,1244899.0,Agawam,Massachusetts,1001.0,1476.0,1986-11-20
3423,33714.0,for_sale,199999.0,3.0,2.0,1.76,1745924.0,Agawam,Massachusetts,1001.0,1968.0,2008-09-19
3430,22188.0,for_sale,419000.0,4.0,2.0,2.00,1417448.0,Pelham,Massachusetts,1002.0,1607.0,2005-07-25
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


## Ajoute les comtés au datasets des trenblements de terre

In [3]:
# Lire ton fichier des villes + comtés
counties = gpd.read_file("data/tl_2021_us_county/tl_2021_us_county.shp")
counties = counties[['GEOID', 'NAME', 'STATEFP', 'COUNTYFP', 'geometry']]
counties = counties.rename(columns={'NAME':'county', 'STATEFP':'state_fips', 'GEOID':'county_fips'})

earthquakes_gdf = gpd.GeoDataFrame(
    filtered_earthquake,
    geometry=gpd.points_from_xy(filtered_earthquake['longitude'], filtered_earthquake['latitude']),
    crs='EPSG:4326'
)

df_with_counties = gpd.sjoin(
    earthquakes_gdf,
    counties[['county_fips','county','state_fips','geometry']],
    how='left',
    predicate='within'
)

filtered_earthquake = df_with_counties

/var/folders/lq/3djnzdbx6ll0000lzlm2th6w0000gn/T/ipykernel_97385/293582311.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  df_with_counties = gpd.sjoin(


## Ajoute les comtés au datasets real-estate

In [5]:
import pandas as pd

# -----------------------------
# 1️⃣ ZIP → COUNTY (ZCTA file)
# -----------------------------
zip_county = pd.read_csv("data/zcta_county_rel_10.txt", dtype=str)

zip_county = zip_county[['ZCTA5', 'STATE', 'COUNTY', 'GEOID']].rename(columns={
    'ZCTA5': 'zip',
    'STATE': 'state_fips',
    'COUNTY': 'county_fips_short',  # 3-digit county code
    'GEOID': 'county_fips'          # FULL 5-digit county code
})

# ZIP → string à 5 digits
zip_county['zip'] = zip_county['zip'].apply(lambda x: str(int(float(x))).zfill(5))


# -----------------------------
# 2️⃣ Nettoyer ZIP dans filtered_realtor
# -----------------------------
filtered_realtor = filtered_realtor.rename(columns={'zip_code': 'zip'})
filtered_realtor['zip'] = filtered_realtor['zip'].apply(lambda x: str(int(float(x))).zfill(5))


# -----------------------------
# 3️⃣ Supprimer anciennes colonnes éventuelles
# -----------------------------
cols_to_drop = ['county_fips', 'state_fips', 'county_fips_short', 'county']
filtered_realtor = filtered_realtor.drop(columns=[c for c in cols_to_drop if c in filtered_realtor.columns])


# -----------------------------
# 4️⃣ ZIP → FIPS (garder premier si ZIP mappe plusieurs counties)
# -----------------------------
zip_major = zip_county.groupby('zip').first().reset_index()


# -----------------------------
# 5️⃣ Merge principal (correct)
# -----------------------------
filtered_realtor = filtered_realtor.merge(
    zip_major[['zip', 'county_fips', 'county_fips_short', 'state_fips']],
    on='zip',
    how='left'
)


# -----------------------------
# 6️⃣ Ajouter le nom du county (depuis shapefile counties)
# -----------------------------
filtered_realtor = filtered_realtor.merge(
    counties[['county_fips', 'county']],
    on='county_fips',
    how='left'
)


# -----------------------------
# 7️⃣ Résultat final
# -----------------------------
filtered_realtor

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip,house_size,prev_sold_date,county_fips,county_fips_short,state_fips,county
0,21163.0,for_sale,525000.0,3.0,3.0,0.45,1813270.0,Agawam,Massachusetts,01001,2314.0,2014-06-25,25013,013,25,Hampden
1,67455.0,for_sale,289900.0,3.0,2.0,0.36,1698080.0,Agawam,Massachusetts,01001,1276.0,2012-10-12,25013,013,25,Hampden
2,97400.0,for_sale,384900.0,3.0,2.0,0.46,1244899.0,Agawam,Massachusetts,01001,1476.0,1986-11-20,25013,013,25,Hampden
3,33714.0,for_sale,199999.0,3.0,2.0,1.76,1745924.0,Agawam,Massachusetts,01001,1968.0,2008-09-19,25013,013,25,Hampden
4,22188.0,for_sale,419000.0,4.0,2.0,2.00,1417448.0,Pelham,Massachusetts,01002,1607.0,2005-07-25,25011,011,25,Franklin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082132,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354,3600.0,2022-03-25,53005,005,53,Benton
1082133,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354,1616.0,2022-03-25,53005,005,53,Benton
1082134,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354,3200.0,2022-03-24,53005,005,53,Benton
1082135,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354,933.0,2022-03-24,53005,005,53,Benton


### Ajoute un date correct (JJ-MM-AAAA) au dataset des trembelements de terre

In [6]:
# 1. Get year, month, day
filtered_earthquake[['year', 'month', 'day']] = (
    filtered_earthquake['date']
    .str.split('-', expand=True)
)

# 2. Fix day (remove time)
filtered_earthquake['day'] = (
    filtered_earthquake['day']
    .str.split(' ', expand=True)[0]
)

# 3. Convert to int (safe)
filtered_earthquake[['year', 'month', 'day']] = (
    filtered_earthquake[['year', 'month', 'day']]
    .apply(pd.to_numeric, errors='coerce')
)

# 4. Drop rows with NaN in date parts
filtered_earthquake = filtered_earthquake.dropna(subset=['year', 'month', 'day'])

# 5. Convert to int after NaN removal
filtered_earthquake[['year', 'month', 'day']] = (
    filtered_earthquake[['year', 'month', 'day']].astype(int)
)

# 6. Drop old date column
filtered_earthquake = filtered_earthquake.drop(columns=['date'])

### Ajoute un date correct (JJ-MM-AAAA) au dataset real-estate

In [7]:
# Convert prev_sold_date to datetime
filtered_realtor["prev_sold_date"] = pd.to_datetime(
    filtered_realtor["prev_sold_date"], errors="coerce"
)

# Extract year / month / day
filtered_realtor["sold_year"] = filtered_realtor["prev_sold_date"].dt.year
filtered_realtor["sold_month"] = filtered_realtor["prev_sold_date"].dt.month
filtered_realtor["sold_day"] = filtered_realtor["prev_sold_date"].dt.day

# Sauvegarder les données

In [34]:
# Save cleaned versions in Parquet
filtered_earthquake.to_parquet('data/earthquake_cleaned.parquet', index=False)
filtered_realtor.to_parquet('data/realtor_cleaned.parquet', index=False)

print("✔ Files saved: earthquake_cleaned.parquet, realtor_cleaned.parquet")

✔ Files saved: earthquake_cleaned.parquet, realtor_cleaned.parquet


# Début de l'aggrégation

In [29]:
### COUNTY + STATE + YEAR AGG ###
# --- Earthquake aggregation by state + county + year ---
agg_eq_state_county_year = (
    filtered_earthquake
    .groupby(['state', 'state_fips', 'county', 'county_fips', 'year'])
    .agg(
        n_earthquakes=('magnitudo', 'count'),
        avg_magnitude=('magnitudo', 'mean'),
        max_magnitude=('magnitudo', 'max'),
        avg_depth=('depth', 'mean'),
    )
    .reset_index()
)

# --- Realtor aggregation by state + county + year ---
agg_re_state_county_year = (
    filtered_realtor
    .groupby(['state', 'state_fips', 'county', 'county_fips', 'sold_year'])
    .agg(
        n_properties=('price', 'count'),
        avg_price=('price', 'mean'),
        median_price=('price', 'median'),
        avg_bedrooms=('bed', 'mean'),
        avg_bathrooms=('bath', 'mean')
    )
    .reset_index()
    .rename(columns={'sold_year': 'year'})
)

### STATE + YEAR AGG ###
# --- Earthquake aggregation by state + year ---
agg_eq_state_year = (
    filtered_earthquake
    .groupby(['state', 'state_fips', 'year'])
    .agg(
        n_earthquakes=('magnitudo', 'count'),
        avg_magnitude=('magnitudo', 'mean'),
        max_magnitude=('magnitudo', 'max'),
        avg_depth=('depth', 'mean')
    )
    .reset_index()
)

# --- Realtor aggregation by state + year ---
agg_re_state_year = (
    filtered_realtor
    .groupby(['state', 'state_fips', 'sold_year'])
    .agg(
        n_properties=('price', 'count'),
        avg_price=('price', 'mean'),
        median_price=('price', 'median'),
        avg_bedrooms=('bed', 'mean'),
        avg_bathrooms=('bath', 'mean')
    )
    .reset_index()
    .rename(columns={'sold_year': 'year'})
)

## Aggrégation

In [30]:
agg_county_year = agg_eq_state_county_year.merge(
    agg_re_state_county_year,
    on=['state', 'state_fips', 'county', 'county_fips', 'year'],
    how='outer'
)

agg_state_year = agg_eq_state_year.merge(
    agg_re_state_year,
    on=['state', 'state_fips', 'year'],
    how='outer'
)
agg_state_year

,state,state_fips,year,n_earthquakes,avg_magnitude,max_magnitude,avg_depth,n_properties,avg_price,median_price,avg_bedrooms,avg_bathrooms
0,Alabama,01,1968,NaN,NaN,NaN,NaN,1.0,89000.000000,89000.0,3.000000,2.000000
1,Alabama,01,1970,NaN,NaN,NaN,NaN,1.0,69000.000000,69000.0,4.000000,1.000000
2,Alabama,01,1972,NaN,NaN,NaN,NaN,2.0,410000.000000,410000.0,4.000000,2.500000
3,Alabama,01,1973,NaN,NaN,NaN,NaN,1.0,874900.000000,874900.0,4.000000,5.000000
4,Alabama,01,1974,NaN,NaN,NaN,NaN,2.0,87450.000000,87450.0,3.500000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3436,Wyoming,56,2019,509.0,1.789568,3.70,3.500413,38.0,735877.631579,397250.0,3.868421,2.684211
3437,Wyoming,56,2020,431.0,1.642599,3.70,3.896056,28.0,512010.714286,393200.0,3.392857,2.392857
3438,Wyoming,56,2021,683.0,1.527496,4.00,4.266750,27.0,444892.296296,325000.0,3.407407,2.111111
3439,Wyoming,56,2022,1739.0,1.052766,3.93,5.173026,321.0,452148.831776,423700.0,3.576324,2.579439


### Correction code FIPS pour les états

In [31]:
# ---------------------------
# FILTER YEARS >= 1990
# ---------------------------
agg_state_year = agg_state_year[agg_state_year["year"] >= 1990].reset_index(drop=True)
agg_county_year = agg_county_year[agg_county_year["year"] >= 1990].reset_index(drop=True)

# ---------------------------
# FIPS TO STR
# ---------------------------
agg_state_year['state_fips'] = (
    agg_state_year['state_fips']
    .astype(str)
    .str.extract(r'(\d+)').iloc[:, 0]
    .fillna('')
    .apply(lambda x: x.zfill(2))
)
agg_county_year['state_fips'] = (
    agg_county_year['state_fips']
    .astype(str)
    .str.extract(r'(\d+)').iloc[:, 0]
    .fillna('')
    .apply(lambda x: x.zfill(2))
)
agg_county_year['county_fips'] = (
    agg_county_year['county_fips']
    .astype(str)
    .str.extract(r'(\d+)').iloc[:, 0]
    .fillna('')
    .apply(lambda x: x.zfill(5))
)


# ---------------------------
# CLEAN MISSING VALUES
# ---------------------------
agg_state_year = agg_state_year.replace(r'^\s*$', np.nan, regex=True)
agg_county_year = agg_county_year.replace(r'^\s*$', np.nan, regex=True)

# Vérification rapide
# agg_state_year
agg_county_year

,state,state_fips,county,county_fips,year,n_earthquakes,avg_magnitude,max_magnitude,avg_depth,n_properties,avg_price,median_price,avg_bedrooms,avg_bathrooms
0,Alabama,01,Autauga,01001,1999,NaN,NaN,NaN,NaN,1.0,324900.0,324900.0,3.00,3.0
1,Alabama,01,Autauga,01001,2000,NaN,NaN,NaN,NaN,1.0,149900.0,149900.0,4.00,4.0
2,Alabama,01,Autauga,01001,2001,NaN,NaN,NaN,NaN,1.0,260000.0,260000.0,4.00,3.0
3,Alabama,01,Autauga,01001,2002,NaN,NaN,NaN,NaN,2.0,324500.0,324500.0,4.50,3.5
4,Alabama,01,Autauga,01001,2003,NaN,NaN,NaN,NaN,4.0,222500.0,212500.0,3.75,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51004,Wyoming,56,Weston,56045,2014,3.0,3.30,3.5,0.0,NaN,NaN,NaN,NaN,NaN
51005,Wyoming,56,Weston,56045,2016,1.0,3.20,3.2,0.0,NaN,NaN,NaN,NaN,NaN
51006,Wyoming,56,Weston,56045,2019,1.0,3.50,3.5,0.0,NaN,NaN,NaN,NaN,NaN
51007,Wyoming,56,Weston,56045,2020,1.0,3.30,3.3,0.0,NaN,NaN,NaN,NaN,NaN


# Sauvegarde des deux aggrégations

In [35]:
# -----------------------------
# Save county + year aggregation
# -----------------------------
agg_county_year.to_parquet("data/agg_county_year.parquet", index=False)
print("✔ Saved county + year aggregation → agg_county_year.parquet")

# -----------------------------
# Save state + year aggregation
# -----------------------------
agg_state_year.to_parquet("data/agg_state_year.parquet", index=False)
print("✔ Saved improved aggregation → agg_state_year.parquet")

✔ Saved county + year aggregation → agg_county_year.parquet
✔ Saved improved aggregation → agg_state_year.parquet
